In [2]:
import pathlib

import requests
import networkx as nx

import dtv_backend.network.build_network_functions as bnf


version = '0.2'
output_dir = pathlib.Path('result')
output_dir = pathlib.Path('~/data/river/dtv/network_digital_twin_v0.2').expanduser()


# Add bathymetry information
TODO: split bathymetry in separate file/notebook
In this section we will make a few adjustments to the network. 
* We will rename the classes to short names for shapefile compatibility
* We will upload the shapefile to GEE and add extra information (bathymetry, maybe later other external variables)
* We will download the new enriched data and continue from there.

In [3]:
# read the networks
network = nx.read_gpickle(output_dir / 'network.pickle')

In [4]:

shp_network = bnf.strip_geometries(bnf.rename_keys(network))


In [6]:
output_dir = pathlib.Path('result')

result_dir =  output_dir / f'network_digital_twin_v{version}-intermediate'

nx.write_shp(shp_network, str(result_dir))
!open $result_dir

/var/folders/fh/tcbrjg6n28b0lzjzh07b5t6m0000gn/T/ipykernel_58265/3332403591.py:5: DeprecationWarning: write_shp is deprecated and will be removed in 3.0.See https://networkx.org/documentation/latest/auto_examples/index.html#geospatial.
  nx.write_shp(shp_network, str(result_dir))
Warning 6: Normalized/laundered field name: 'length' to 'length_1'


# Add bathymetry info
Now, for now, a manual step. You can now upload the files in the folder above to the google earthengine. At the google earthengine all the [bathymetry](https://maps.rijkswaterstaat.nl/geoweb55/index.html?viewer=Bathymetrie_Nederland) files exists . These can be intersected using the script [fis-bathy](https://code.earthengine.google.com/d9250d62af8146d05cab278aa83df877). This script will result in a new file called [edges.geojson](https://storage.googleapis.com/slr/bodemhoogte/edges.geojson). This file will be used from here.


In [4]:
# assuming that the above succeeded, we have a network with extra info (stored in geojson)
resp = requests.get('https://storage.googleapis.com/slr/bodemhoogte/edges.geojson')
extra_info = resp.json()

In [5]:
# extend network with extra info
for feature in extra_info['features']:
    start_id = feature['properties']['start-id']
    end_id = feature['properties']['end-id']
    # lookup the existing edge
    edge = network.edges[(start_id, end_id)]
    # determine which keys to add
    extra_keys = [
        key 
        for key 
        in feature['properties'].keys() 
        if key.startswith('lat') or key.startswith('nap')
    ]
    for key in extra_keys:
        edge[key] = feature['properties'][key] 


NameError: name 'network' is not defined